In [1]:
import pretrainedmodels
import torch
from torchvision import transforms
from PIL import Image

print(pretrainedmodels.model_names)

In [3]:
model_name = 'nasnetamobile' 
nasnetamobile = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

# Save the model
torch.save(nasnetamobile, './Saved-Model/NASNet-Mobile.pt')
saved_nasnetamobile = torch.load('./Saved-Model/NASNet-Mobile.pt')

In [4]:
#Prepare data for inference

transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

#Line [1]: Here we are defining a variable transform which is a combination of all the image transformations to be carried out on the input image.

#Line [2]: Resize the image to 256×256 pixels.

#Line [3]: Crop the image to 224×224 pixels about the center.

#Line [4]: Convert the image to PyTorch Tensor data type.

#Line [5-7]: Normalize the image by setting its mean and standard deviation to the specified values.

img = Image.open("../../data/elephant.jpg")
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

In [6]:
nasnetamobile.eval()
saved_nasnetamobile.eval()
out = nasnetamobile(batch_t)
out_ = saved_nasnetamobile(batch_t)
print(out.shape)

torch.Size([1, 1000])


In [7]:
with open('../../ImageNet_classes/imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

"""
_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())
"""
print('******* from package **********')
#print the top 5 classes predicted by the model
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]])

print('******* saved model **********')
#print the top 5 classes predicted by the model
_, indices = torch.sort(out_, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]])

******* resnet50 from package **********
[('Indian elephant, Elephas maximus', 93.8409194946289), ('tusker', 2.3833913803100586), ('African elephant, Loxodonta africana', 1.9009809494018555), ('hippopotamus, hippo, river horse, Hippopotamus amphibius', 0.6176109313964844), ('triceratops', 0.4168368875980377)]
******* resnet50 saved model **********
[('Indian elephant, Elephas maximus', 93.8409194946289), ('tusker', 2.3833913803100586), ('African elephant, Loxodonta africana', 1.9009809494018555), ('hippopotamus, hippo, river horse, Hippopotamus amphibius', 0.6176109313964844), ('triceratops', 0.4168368875980377)]
